In [1]:
import numpy as np
import math

def zxy_to_xy(z, x, y, tvec, euler, K, D):
    fx, fy, cx, cy = K
    k1, k2, p1, p2 = D
    
    xyz_e_mat = get_xyz_euler(x, -z, y, tvec, euler)
    xy_u_mat = get_xy_u(xyz_e_mat)
    sqr = get_sqr(xy_u_mat)
    xy_d_mat = get_xy_d(sqr, xy_u_mat, k1, k2, p1, p2)
    xy_p_mat = get_xy_p(xy_d_mat, fx, fy, cx, cy)
    return np.array([ [xy_p_mat[0][0], xy_p_mat[1][0]]   ])

def get_xyz_euler(x, y, z, tvec, euler):
    # tx, ty, tz = tvec
    tx, tz, ty = tvec
    # ty, tx, tz = tvec
    # ty, tz, tx = tvec
    # tz, ty, tz = tvec
    # tz, tz, ty = tvec
    
    
    # ud, lr, rot = euler
    # ud, rot, lr = euler
    lr, ud, rot = euler
    # lr, rot, ud = euler
    # rot, lr, ud = euler
    # rot, ud, lr = euler
    
    
    op1 = np.asarray([
        [0, 1, 0],
        [0, 0, -1],
        [-1, 0, 0]
    ])
    op2 = np.asarray([
        [1, 0, 0],
        [0, math.cos(rot), -math.sin(rot)],
        [0, math.sin(rot), math.cos(rot)]
    ])
    op3 = np.asarray([
        [math.cos(ud), 0, math.sin(ud)],
        [0, 1, 0],
        [-math.sin(ud), 0, math.cos(ud)]
    ])
    op4 = np.asarray([
        [math.cos(lr), -math.sin(lr), 0],
        [math.sin(lr), math.cos(lr), 0],
        [0,0, 1]
    ])
    op5 = np.asarray([
        [x - tx],
        [y - ty],
        [z + tz]
    ])
    ret = np.matmul(op1, op2)
    ret = np.matmul(ret, op3)
    ret = np.matmul(ret, op4)
    ret = np.matmul(ret, op5)
    return ret


def get_xy_u(xyz_e_mat):
    xe = xyz_e_mat[0][0]
    ye = xyz_e_mat[1][0]
    ze = xyz_e_mat[2][0]
    if ze !=0:
        return np.asarray([[xe/ze],[ye/ze]])
    else:
        return np.asarray([[ 0 ],[ 0 ]])
    


def get_sqr(xy_u_mat): ##// (2,1)
    xu = xy_u_mat[0][0]
    yu = xy_u_mat[1][0]
    return xu*xu + yu*yu 
    

def get_xy_d(sqr, xy_u_mat, k1, k2, p1, p2):
    op1_coeff = 1 + k1*sqr + k2*sqr*sqr
    xu = xy_u_mat[0][0]
    yu = xy_u_mat[1][0]
    op1 = np.asarray([
        [xu * op1_coeff],
        [yu * op1_coeff]
    ])
    op2 = np.asarray([
        [2*p1*xu*yu + p2*(sqr + 2*xu*xu)],
        [p1*(sqr + 2*yu*yu) + 2*p2*xu*yu]
    ])
    return op1 + op2
    

def get_xy_p(xy_d_mat, fx, fy, cx, cy):
    xd = xy_d_mat[0][0]
    yd = xy_d_mat[1][0]
    
    op1 = np.asarray([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 0]
    ])
    op2 = np.asarray([
        [xd],
        [yd],
        [1]
    ])
    
    ret = np.matmul(op1, op2)
    
    return np.asarray( [[ ret[0][0] ],[ ret[1][0] ]] )

def parse_cal(conf_pth):
    with open(conf_pth, 'r') as f:
        f.readline()
        conf_lidar = yaml.load(f, Loader=yaml.FullLoader)
        fx = float(conf_lidar['fx'])
        fy = float(conf_lidar['fy'])
        cx = float(conf_lidar['cx'])
        cy = float(conf_lidar['cy'])
        k1 = float(conf_lidar['k1'])
        k2 = float(conf_lidar['k2'])
        p1 = float(conf_lidar['p1/k3'])
        p2 = float(conf_lidar['p2/k4'])
        euler = conf_lidar["rvec"]
        tvec = conf_lidar['translation_vector']
    K = [fx, fy, cx, cy]
    D = [k1, k2, p1, p2]
    
    # for i in range(len(tvec)):
    #     tvec[i] = -0.25*tvec[i]
        
    return K, D, euler, tvec



def iter_pts(pcd_arr, K, D, euler, tvec):
    ret = np.zeros(shape=(1,2))
    depth = []
    for i in range(len(pcd_arr)):
        x = pcd_arr[i][0]
        y = pcd_arr[i][1]
        z = pcd_arr[i][2]
        d = math.sqrt(x**2 + y**2)
        depth.append(d)
        xy_2d = zxy_to_xy(z, x, y, tvec, euler, K, D)
        print(xy_2d)
        if ret.all() == 0:
            ret = xy_2d
        else:
            ret = np.concatenate((ret, xy_2d), axis=0)
    
    return ret, depth

def proc_2d(projected_arr):
    for i in range(len(projected_arr)):
        x = projected_arr[i][0]
        y = projected_arr[i][1]
        ret_pt = utils.rotate_2d([x,y], [960,600], 90)
        
        projected_arr[i][0] = ret_pt[0] -25 
        projected_arr[i][1] = ret_pt[1] 
        
                
# K, D, euler, tvec = parse_cal(r"C:\workspace\calib_48\conf_a.yaml")
# pcd_path = r"C:\aivill_sey_ryeong\labeling\라벨링툴\구조변경test\A_Clip_00238_06\Lidar/2-048_00238_LR_010.pcd"
# img_path = r"C:\aivill_sey_ryeong\labeling\라벨링툴\구조변경test\A_Clip_00238_06\Camera\CameraFront\blur/2-048_00238_CF_010.png"
# out_path = "./"
# direction = 1


# pcd_arr = calib_parser.parse_pcd(pcd_path)
# pcd_arr = visualizer.cut_pcd(pcd_arr, direction)
# projected_arr, depth = iter_pts(pcd_arr, K, D, euler, tvec)
# proc_2d(projected_arr)
# visualizer.make_img_lidar(img_path, out_path, projected_arr, depth)